In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
import canny_edge_detector as canny
from sobel import show_images
import matplotlib.pyplot as plt 
import skimage.io as io
import os

import numpy as np
import cv2


paths=[]
# def load_data(dir_name = 'images'):    
#     imgs = []
#     for filename in os.listdir(dir_name):
#         if os.path.isfile(dir_name + '/' + filename):
#             img = io.imread(dir_name + '/' + filename)
#             img = rgb2gray(img)
#             imgs.append(img)
#             paths.append(dir_name + '/' + filename)
#     return imgs
# def visualize(imgs, format=None, gray=False):
#     plt.figure(figsize=(20, 40))
#     for i, img in enumerate(imgs):
#         if img.shape[0] == 3:
#             img = img.transpose(1,2,0)
#         plt_idx = i+1
#         plt.subplot(2, 2, plt_idx)
#         plt.imshow(img, format)
#     plt.show()
# imgs = load_data()



# lower = cv2.threshold(img, 40, 255, cv2.THRESH_BINARY )[0]/3200
# upper = lower/3200+.0499


o_circles,inner_circles = canny.process_for_daugman(canny)
print(o_circles,inner_circles)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[(132, 131, 57)] [(132, 131, 26)]
